# Project 4: **Build a Deep Research System**
Welcome to project 4! For this project, we shift our focus from tool use and agents to *reasoning* models. You will practice state‑of‑the‑art inference‑time scaling methods such as *Chain‑of‑Thought* prompting and *Tree‑of‑Thoughts*, and briefly explore high-levels of training reasoning models using techniques like **STaR**.


Finally, you will put everything together to build a *deep research agent* that can browse the web, reason over what it finds, and give structured answers.

## Learning Objectives  
* Apply common inference‑time scaling methods: **zero‑shot / few‑shot CoT, self‑consistency, sequential decoding, tree‑of‑thoughts**  
* Gain intuition for **training** reasoning‑capable models following **STaR** approach 
* Build a minimal **deep‑research agent** that combines step‑by‑step reasoning with live web search   
* Practice extending deep-search to a multi-agent system 

## Roadmap  
1. Environment setup  
2. Inference‑time scaling  
   2.1 Few‑shot & zero‑shot CoT  
   2.2 Self‑consistency
   2.3 Sequential revisions  
   2.4 Tree‑of‑Thought
3. STaR for training models for reasoning  
4. Deep-research agent  
5. (Optional) Multi-agent deep-research

# 1‑ Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yaml && conda activate deep_research

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=deep_research --display-name "deep_research"
```
Once this is done, you can select "deep_research" from the Kernel → Change Kernel menu in Jupyter or VS Code.

## 1.2 Ollama setup

In this project we use the `llama3.2:3b` and `deepseek-r1:8b` models. You can try other smaller or larger reasoning LLMs such as `qwen2.5:3b-instruct` or `phi4-mini` to compare performance. Explore available models here: https://ollama.com/library.

```bash
ollama pull llama3.2:3b
ollama pull deepseek-r1:8b
# Additional small reasoning models to compare
# ollama pull qwen2.5:3b-instruct
# ollama pull phi4-mini

```

`ollama pull` downloads the model so you can run it locally without API calls.

---  
# 2‑ Inference‑time scaling

Inference-time scaling refers to techniques that make an existing model reason better without retraining it. Instead of changing the model’s weights, we achieve reasoning capability by adjusting how we prompt, sample, or aggregate LLM's outputs.

In this section, we’ll explore several inference-time strategies that improve reasoning quality using a non-reasoning base model. You will experiment with and compare methods such as:

- Few-shot Chain-of-Thought (CoT)
- Zero-shot CoT
- Self-consistency
- Sequential revision
- Tree-of-Thoughts (ToT)

### 2.1: Few‑Shot CoT
Few-shot prompting helps a model reason by showing one or multiple examples before asking a new question. By observing the pattern of reasoning and final answers, the model learns how to structure its own reasoning process on the new input.

In this exercise, you will create a prompt that includes a few example Q&A pairs demonstrating step-by-step reasoning. Then, you will feed a new question and see the model’s output.

In [ ]:
# Step 1: Write a few examples showing reasoning steps
# Step 2: Write your new question
# Step 3: Concatenate examples + new question into a single prompt
# Step 4: Call your Ollama or OpenAI client to get a response from llama3.2:3b # e.g., client.chat.completions.create(...)
# Step 5: Print the final answer

from openai import OpenAI

client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")
MODEL = "llama3.2:3b"

examples = [
    (
        "Question: If I buy 2 books for $5 each, how much do I spend?\n"
        "Reasoning: Each book costs $5, so 2 * $5 = $10.\n"
        "Answer: $10"
    ),
    (
        "Question: A jar has 12 candies and I eat 3. How many remain?\n"
        "Reasoning: Starting with 12 candies and removing 3 leaves 9.\n"
        "Answer: 9"
    ),
]
newline = chr(10)
new_question = "Question: Tom walks 3 miles before lunch and 2 more miles after lunch. How far did he walk in total?"
prompt = (newline * 2).join(examples + [f"{new_question}{newline}Reasoning:"])

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a meticulous tutor who explains each step before giving the answer."},
        {"role": "user", "content": prompt},
    ],
)

final_answer = response.choices[0].message.content.strip()
print(final_answer)


### (Optional) Few-shot CoT on GPT2
GPT-2 is a pre-trained language model without instruction tuning. It continues text rather than answering questions. In this section, you'll try the exact same CoT pattern on GPT-2 and observe what happens. The goal is to test whether few-shot CoT alone can elicit structured reasoning from a non-chat LLM.

In [ ]:
import os
import torch
from transformers import pipeline

# Step 1: Load GPT-2 text-generation from huggingface (https://huggingface.co/docs/transformers/en/model_doc/gpt2)
# Step 2: Write 1–2 few-shot reasoning examples (short, explicit steps + final answer in your own unique format)
# Step 3: Append a new test question after the examples to form one prompt string
# Step 4: Generate 1–3 completions with different decoding settings (e.g., greedy vs. top-k)
# Step 5: Print raw outputs; check if steps are followed and if the final answer is correct

device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text-generation", model="gpt2", device=device)

examples = """Example 1:
Question: What is 7 minus 2?
Steps:
1. Start with 7.
2. Subtract 2 to get 5.
Answer: 5

Example 2:
Question: If a box holds 4 apples and you add 3 more, how many apples are there?
Steps:
1. Begin with 4 apples.
2. Add 3 apples to reach 7.
Answer: 7

"""
question = "Question: A baker makes 5 loaves in the morning and 4 in the afternoon. How many loaves were made in all?
Steps:"
prompt = examples + question

settings = [
    {"name": "greedy", "params": {"max_new_tokens": 60, "do_sample": False}},
    {"name": "top_k", "params": {"max_new_tokens": 60, "do_sample": True, "top_k": 40, "temperature": 0.9}},
    {"name": "top_p", "params": {"max_new_tokens": 60, "do_sample": True, "top_p": 0.9, "temperature": 0.8}},
]

for setting in settings:
    outputs = generator(prompt, **setting["params"])
    print(f"--- {setting['name']} ---")
    print(outputs[0]["generated_text"][len(prompt):].strip())


### 2.2: Zero‑Shot Chain‑of‑Thought
Zero-shot CoT encourages the model to reason without examples by adding a short cue such as “Let’s think step by step.” This simple phrase often activates the model’s latent reasoning ability even when no demonstrations are provided. It serves as a baseline to compare with few-shot and other inference-time scaling methods.

In [ ]:
# Step 1: Write the question and a zero-shot CoT cue (e.g., "Let's think step by step.")
# Step 2: Build a single prompt string that includes brief role guidance plus the question
# Step 3: Call your Ollama or OpenAI client to get a response from llama3.2:3b  # e.g., client.chat.completions.create(...)
# Step 4: Print the chain and the final answer

question = "What is the smallest prime number greater than 20?"
prompt = f"""You are a logical assistant. Provide a concise chain of thought before answering.
Question: {question}
Let's think step by step."""
response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
)
print(response.choices[0].message.content.strip())


### 2.3 Self‑Consistency
Self-consistency enhances reasoning accuracy by sampling multiple independent reasoning paths for the same question instead of relying on a single deterministic answer. Each run may follow a slightly different logical chain, and the diversity helps correct individual mistakes. After generating several reasoning traces, you then aggregate the final answers using majority voting.

This approach is especially useful when tasks involve multi-step reasoning or arithmetic, where single-path outputs may be incorrect.

In [ ]:
from openai import OpenAI
import re, collections

client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")
MODEL = "llama3.2:3b"

def cot_answer(question, temperature=1.0):
    # Generate a step-by-step reasoning chain for the given question and extract the final answer.
    response = client.chat.completions.create(
        model=MODEL,
        temperature=temperature,
        messages=[
            {"role": "system", "content": "Think carefully, provide a numbered reasoning chain, and end with 'Final answer: <value>'."},
            {"role": "user", "content": f"""Question: {question}
Let's reason step by step."""},
        ],
    )
    chain = response.choices[0].message.content.strip()
    match = re.search(r"Final answer:\s*(.*)", chain, re.IGNORECASE)
    final_answer = match.group(1).strip() if match else chain.splitlines()[-1].strip()
    return final_answer, chain

def self_consistent(question, n=10):
    # Run multiple reasoning chains and select the most frequent final answer by majority voting.
    counter = collections.Counter()
    for i in range(n):
        temperature = 0.7 + (i * 0.05)
        final_answer, chain = cot_answer(question, temperature=temperature)
        counter[final_answer] += 1
    winner, _ = counter.most_common(1)[0]
    return winner, counter


question = "What is the square root of 144?"
winner, counter = self_consistent(question)
print("Votes:", counter)
print("Chosen answer:", winner)


### 2.4: Sequential Revision

Sequential revision iteratively improves an answer by generating a first draft, critiquing it, and producing revised drafts that condition on prior answers. Each round should be short and focused, so improvements accumulate without drifting from the question.

In [ ]:
MODEL = "llama3.2:3b"

def sequential_revision(question: str, max_steps: int = 3) -> str:
    # Generate an initial draft answer, then iteratively refine it by conditioning each revision on the previous one.
    # Step 1: Ask the model to produce the first draft for the given question
    # Step 2: Loop for max_steps-1 times, each time feeding the last draft back to the model with a request to revise
    # Step 3: Print each draft to observe how the answer evolves
    # Step 4: Return the final improved draft
    drafts = []
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Draft a thoughtful answer to the question."},
            {"role": "user", "content": f"Question: {question}\nProvide an initial draft answer."},
        ],
    )
    drafts.append(response.choices[0].message.content.strip())
    print(f"Draft 1:\n{drafts[-1]}\n")

    for step in range(1, max_steps):
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "Improve the given draft keeping the reasoning explicit and factual."},
                {
                    "role": "user",
                    "content": (
                        f"Original question: {question}\n"
                        f"Previous draft:\n{drafts[-1]}\n"
                        "Revise the draft for clarity, correctness, and completeness."
                    ),
                },
            ],
        )
        drafts.append(response.choices[0].message.content.strip())
        print(f"Draft {step + 1}:\n{drafts[-1]}\n")

    return drafts[-1]


# Step 1: Define a question that benefits from multi-step reasoning
# Step 2: Call sequential_revision(question, max_steps)
# Step 3: Print the final output
question = "Explain how photosynthesis converts sunlight into energy for plants."
final_response = sequential_revision(question, max_steps=3)
print("Final answer:\n", final_response)


### 2.5 Tree‑of‑Thoughts
Tree-of-Thoughts reframes reasoning as a search process rather than a single forward chain.
Instead of producing one linear sequence of thoughts, the model generates multiple candidate thoughts at each step, evaluates their promise, and then expands only the best few. This allows exploration of different reasoning paths before committing to a final answer, similar to how humans brainstorm, prune, and refine ideas.


In this section, you’ll experiment with two simplified versions of ToT:
1. Word Ladder puzzle solver: a small example where each “thought” is a candidate word transition.
2. Generic ToT search (depth 2, width 2): a minimal logic to expand, evaluate, and select reasoning branches

In [ ]:
###### Word Ladder Puzzle ##########

def neighbors(word, vocabulary):
    # Generate all valid one-letter mutations of 'word' that exist in 'vocabulary' and return them.
    neighbors_list = []
    for candidate in vocabulary:
        if len(candidate) != len(word) or candidate == word:
            continue
        differences = sum(c1 != c2 for c1, c2 in zip(word, candidate))
        if differences == 1:
            neighbors_list.append(candidate)
    return neighbors_list


def tree_of_thought(start, goal, vocab, max_depth=5, beam_width=4):
    # Search over partial thoughts (paths) using a small beam.
    # Step 1: Initialize the frontier with a single path [start]
    # Step 2: For each depth, expand each path by one neighbor from 'neighbors'
    # Step 3: Score paths by edit distance between last word and 'goal' (smaller is better)
    # Step 4: Keep the top 'beam_width' paths and stop early if any reaches 'goal'
    # Step 5: Return the best goal-reaching path or None
    def edit_distance(a, b):
        dp = [[0] * (len(b) + 1) for _ in range(len(a) + 1)]
        for i in range(len(a) + 1):
            dp[i][0] = i
        for j in range(len(b) + 1):
            dp[0][j] = j
        for i in range(1, len(a) + 1):
            for j in range(1, len(b) + 1):
                cost = 0 if a[i - 1] == b[j - 1] else 1
                dp[i][j] = min(
                    dp[i - 1][j] + 1,
                    dp[i][j - 1] + 1,
                    dp[i - 1][j - 1] + cost,
                )
        return dp[-1][-1]

    frontier = [[start]]

    for depth in range(max_depth):
        new_paths = []
        for path in frontier:
            current = path[-1]
            if current == goal:
                return path
            for nxt in neighbors(current, vocab):
                if nxt in path:
                    continue
                new_paths.append(path + [nxt])
        if not new_paths:
            break
        new_paths.sort(key=lambda p: edit_distance(p[-1], goal))
        frontier = new_paths[:beam_width]
        for path in frontier:
            if path[-1] == goal:
                return path

    for path in frontier:
        if path[-1] == goal:
            return path
    return None


vocab = {"hit", "dot", "cog", "log", "dog", "lot", "lit", "hot"}
print(tree_of_thought("hit", "cog", vocab)) # one candidate solution: ['hit', 'hot', 'dot', 'dog', 'cog']


In [ ]:
###### Generic ToT Search ##########

import re

MODEL = "llama3.2:3b"

def propose_thoughts(question, state, k=2):
    # Propose up to k next “thoughts” that extend the current partial solution/state.
    # Steps: build a short prompt with problem + current state; call your client with n=k. Then return a list of stripped strings (≤ k).
    prompt = f"""You are brainstorming answers to a problem.
Problem: {question}
Current state: {state or 'Start'}
Suggest the next step or idea."""
    response = client.chat.completions.create(
        model=MODEL,
        n=k,
        messages=[{"role": "user", "content": prompt}],
    )
    thoughts = []
    for choice in response.choices:
        thought = choice.message.content.strip()
        if thought and thought not in thoughts:
            thoughts.append(thought)
    return thoughts[:k]


def score_state(question, state):
    # Score how promising a partial solution is on a 1–10 scale (higher is better).
    # Steps: build a rating prompt; call the model; parse the first integer 1–10;
    prompt = f"""Rate the promise of the partial solution on a scale from 1 to 10, where 10 is an excellent, nearly complete plan.
Problem: {question}
Partial solution: {state or 'No steps yet'}
Respond with only an integer."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
    )
    content = response.choices[0].message.content
    match = re.search(r"(10|[1-9])", content)
    return int(match.group(1)) if match else 1


def tree_of_thoughts(question, depth=2, width=2):
    # Run a tiny ToT search: expand states with propose_thoughts, score with score_state, keep top-k at each depth.
    # Steps: initialize frontier=[("", 0)]; for each depth, expand each state with k=width thoughts; score each; sort by score desc; keep top 'width'; return best state and score.
    frontier = [("", 0)]
    best_state, best_score = "", -1

    for _ in range(depth):
        candidates = []
        for state, _ in frontier:
            thoughts = propose_thoughts(question, state, k=width)
            for thought in thoughts:
                new_state = (state + "\n" + thought).strip()
                score = score_state(question, new_state)
                candidates.append((new_state, score))
                if score > best_score:
                    best_state, best_score = new_state, score
        if not candidates:
            break
        candidates.sort(key=lambda item: item[1], reverse=True)
        frontier = candidates[:width]

    return best_state, best_score


question = "Design a plan for a weekend science workshop for 12-year-olds."
solution, score = tree_of_thoughts(question)

print(f"Best solution (score {score}):
{solution}")


---  
# 3‑ Training Models for Reasoning

### 3.1: CoT Training
Chain-of-Thought (CoT) training conditions the model on explicit rationales during fine-tuning. Instead of teaching the model to output only the final answer, we train on (question, rationale, answer) so the model learns to internalize multi-step reasoning patterns. A practical recipe is STaR (Self-Taught Reasoner), which uses a stronger teacher model to bootstrap rationales that a smaller student can learn from.

For tasks that require multi-hop reasoning, models fine-tuned on rationales often achieve higher accuracy and are more stable at inference time than models trained on direct answers only. 

Training a full language model is beyond the scope of this notebook, but here is the high-level workflow followed by a short pseudocode:
- Collect questions: Prepare a dataset of questions and correct answers.
- Generate rationales: Use a strong LLM to produce step-by-step reasoning ending with the correct answer.
- Filter and clean: Discard incorrect or low-quality rationales.
- Prepare training data: Format triples (question, rationale, answer) for supervised fine-tuning.
- Fine-tune: Fine-tune the LLM on rationales.
- Iterate: Refine prompts, improve data quality, and retrain for stronger reasoning.

In [ ]:
# Pseudocode (STaR loop)
# for round in 1 ... iters:
    # STEP 1: self-generate reasoning (teacher creates rationale + answer)
    # STEP 2: keep only correct, high-quality traces
    # STEP 3: fine-tune student on (question, rationale, answer) data

### 3.2: ORM vs PRM + RL
Training a Reward Model (RM) allows large language models to be improved through reinforcement learning (RL). Instead of fine-tuning directly on examples, we train a separate model that can score or rank model outputs, and use those scores as feedback signals to refine the policy model.

Two main reward modeling approaches are ORM (predicts a scalar reward for the final answer) and PRM (evaluates the reasoning steps instead of just the outcome)



| Approach | Typical loss | When to use |
|-----------|-------------|-------------|
|*Outcome Reward Model* | Predict scalar reward | Easy to collect training data using verifiers |
|*Process Reward Model* | Predict rewards per step | Difficult to collect training data but more accurate |
| *RLHF* | Use RM as reward in **RL** fine‑tuning | Aligns policy with human signals | Aligns model policy with human or synthetic preferences




In [ ]:
# for round = 1 ... iters:
    # STEP 1:  Generate reasoning
        # sample a minibatch of questions
        # policy roll‑out (actions + log‑probs)
    # STEP 2:  Score the trajectory
        # ORM: scalar reward for the final answer / PRM: scalar reward for the thought process
    # STEP 3:  Reinforce the policy (PPO)

---  
# 4‑ A Deep Research Agent

A deep-research agent pairs a reasoning model (e.g., deepseek-r1) with external tools for web search and retrieval. We will follow the ReAct pattern: the model writes short thoughts, decides when to call tools, reads observations, and continues reasoning until it can answer or reaches a step limit.

We now combine a **search tool** with a reasoning model (e.g., `deepseek-r1`) in a multi-step setup. We follow the *ReAct* pattern (reason → tool → observation):

1. The model reasoins and decides to use tools
2. The agent searches and feed condensed snippets back as context
3. Iterate until the model answers or hits a step limit

We use `AgentType.OPENAI_FUNCTIONS`, which hides the loop inside the LangChain agent.

In [ ]:
from ddgs import DDGS
from langchain.tools import Tool


def ddg_search(query: str, k: int = 5) -> str:
    # Use DDGS to run a simple web search and return joined snippets.
    snippets = []
    with DDGS() as ddgs:
        for result in ddgs.text(query, max_results=k):
            title = result.get("title", "")
            body = result.get("body") or result.get("snippet") or ""
            snippet = f"{title}: {body}".strip(": ")
            if snippet:
                snippets.append(snippet)
    return "\n".join(snippets)


search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search,
    description="Search the public web. Input: a plain English query. Returns: concatenated snippets.",
)


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama

MODEL = "deepseek-r1:8b"
question = "What are the best resources to learn machine learning in 2025?"

# Step 1: Initialize the reasoning model via ChatOllama
llm = ChatOllama(model=MODEL)

# Step 2: Build the agent with tool access (DuckDuckGo Search) and function-calling interface (initialize_agent)
agent = initialize_agent([search_tool], llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)


# Step 3: Ask a query and let the agent search + reason to produce an answer
result = agent.invoke({"input": question})
print(result.get("output", result))


# Optional (Multi-agent Deep Research)
Instead of a single multi-step agent, you can design multiple collaborating agents such as a Planner, Searcher, Summarizer, and Verifier that pass information and refine each other’s outputs. This setup improves robustness, diversity of reasoning, and division of labor.

Try building a simple setup with 2–3 agents that share goals and messages, for example Planner → Researcher → Writer.

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def parallel_research(query, n=3):
    # Run n independent research runs in parallel and return their answers.
    # Steps: use ThreadPoolExecutor; submit n calls to your agent/search pipeline; gather results in order.
    with ThreadPoolExecutor(max_workers=n) as executor:
        futures = [executor.submit(agent.invoke, {"input": query}) for _ in range(n)]
        outputs = []
        for future in futures:
            try:
                result = future.result()
                if isinstance(result, dict):
                    outputs.append(result.get("output", str(result)))
                else:
                    outputs.append(str(result))
            except Exception as exc:
                outputs.append(f"Error: {exc}")
    return outputs


answers = parallel_research("What are the best resources to learn ML in 2025?")
for i, a in enumerate(answers, 1):
    print(f"[Run {i}] {a[:200]}…")


## 🎉 Congratulations!

* Practised various inference‑time reasoning methods
* Gained intuition about training reasoning models
* You have built a **deep-research agent**: reasoning model like deep-seek r1 + ReAct-style agent + tool use (web search)
* Try adding more tools, and extending the deep-research to a multi-agent system: many agents researching web in parallel.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.